In [1]:
import pandas as pd
import re

In [2]:
arquivo = pd.read_excel('la_prima_fevereiro2025.xlsx')

In [3]:
# Aqui eu seleciono todas as colunas que eu preciso

df_arquivo = arquivo[['Data da venda','Título do anúncio', 'SKU', 'Unidades', 'Receita por produtos (BRL)', 'Receita por envio (BRL)', 'Tarifa de venda e impostos', 'Tarifas de envio', 'Estado']]

In [4]:
# Coloco o index em ordem decrecente pq o ML não deixa eu fazer um sort_by...

df_arquivo = df_arquivo.sort_index(ascending=False)

In [5]:
# Reseto o index 

df_arquivo = df_arquivo.reset_index(drop=True)

In [6]:
# Excluo as linhas que marcam as vendas como cancelado ou cancelada 

df_arquivo = df_arquivo[~df_arquivo['Estado'].str.contains('cancelado|cancelada', case=False, na=False)]

In [7]:
# Extraio do título do anuncio as marcas e substituo. Deixando só as marcas.

df_arquivo['Título do anúncio'] = df_arquivo['Título do anúncio'].str.extract(r'(Orient|Lince|Champion|Seculus|Mondaine|Technos|X-Games|X Games|Casio|Skmei|Herweg|X-watch|X watch)', flags=re.IGNORECASE).fillna(df_arquivo['Título do anúncio'])

In [8]:
# Substituo os valores nulos por "0" para poder fazer a soma das colunas

df_arquivo['Receita por envio (BRL)'] = df_arquivo['Receita por envio (BRL)'].fillna(0)
df_arquivo['Tarifas de envio'] = df_arquivo['Tarifas de envio'].fillna(0)

In [9]:
# Crio uma nova coluna, somando as duas colunas 

df_arquivo['Tarifa de Frete'] = df_arquivo['Receita por envio (BRL)'] + df_arquivo['Tarifas de envio']

In [10]:
# Transformo a coluna de negativos em positivos para facilitar o relatorio

df_arquivo['Tarifa de Frete'] = df_arquivo['Tarifa de Frete'].abs()

In [11]:
df_arquivo["Tarifa de venda e impostos"] = df_arquivo["Tarifa de venda e impostos"].abs()

In [12]:
preco_custo = {
    "6103": 13.88, "6102": 17.21, "2612": 14.56, "F-91W-1DG-SC": 45.84, "MBSS1004 B1SX": 88.03,
    "MBSS1154A E2SX": 118.84, "MBSS1154A P2SX": 153.04, "MBSS1154A S2SX": 118.84, "F-94WA-8DG-SC": 45.40,
    "MBSS1154A G2SX": 118.84, "MBSS1155A P2SX": 153.04, "MBSS1155A G2SX": 118.84, "MBSS1155A S2SX": 118.84,
    "GA-700-1BDR": 301.29, "MBSS1154A D2SX": 118.84, "GA-700-1ADR": 234.01, "MBSS1155A D2SX": 127.39,
    "2976": 53.64, "MDN4596L PXPX": 128.21, "MPSS1019 P2PX": 161.60, "MBSS1360 P2SX": 127.39,
    "MGSS1180 P2KX": 153.04, "CH40080T": 117.23, "CH40080V": 126.26, "MDG4619L BXKX": 128.20,
    "MDM4619L BXSX": 111.14, "MDR4619L BXRX": 128.20, "CH50006K": 210.78, "CH50006P": 239.99,
    "CH50006Q": 239.99, "CH50006T": 210.78, "CH50006V": 210.78, "CH50006W": 210.78, "MBSS1004A B1SX": 153.04,
    "MBSS1381 P2SX": 118.84, "MBSS1004A P1SX": 153.04, "CN24351N": 135.37, "FGSS2004 B2KX": 170.14,
    "469EC7F C1KX": 409.54, "469WC2F B1KX": 366.79, "469WC2F C1KX": 366.79, "469TT043F S1SK": 452.30,
    "469WA3F A1SX": 366.79, "469WA3F E1SX": 366.79, "469WC1F B1SK": 366.80, "469EC7F B1KX": 409.54,
    "MOLIFEAC/8K": 272.28, "MBSS1360 D2SX": 153.04, "32193GPMVDH2": 113.52, "44016GPSVBA2": 138.60,
    "44016GPSVDA1": 138.51, "MOLIFEAI/7P": 311.23, "LRR4724L KP62S2RX": 84.99, "469ED1F S1KS": 409.54,
    "MOLIFEAK/8L": 284.32, "CN27732W": 120.01, "A11327": 48.00, "469TT043F P1SK": 452.30,
    "469WC2F P1KX": 366.80, "MOLIFEAJ/8A": 272.28, "MOLIFEAL/7K": 311.23, "MOLIFEAN/8R": 268.62,
    "MOLIFEAO/8C": 268.62, "SK40100": 48.00, "SK40156": 47.00, "20943LPSKDS1K1": 178.75,
    "44056GPSKDA1K1": 176.99, "20942LPSKDS1K1": 178.75, "44026GPSKDA1K1": 178.76, "44054LPSKDA1K1": 178.76,
    "20939LPSKDS1K1": 176.98, "44025GPSKDA1K1": 192.38, "20941LPSKDS1K1": 176.98, "44027GPSKDA1K1": 176.98,
    "44078GPSKDA1K1": 178.91, "44040LPSKDA1K1": 178.91, "20967LPSKDA1K1": 178.91, "35019LPSKDA1K1": 178.91,
    "44115GPSVDA1": 138.89, "44116GPSVDA2": 138.60, "23701LPSKDA1": 178.91, "23700GPSKDA1": 178.91,
    "44111GPSKDA1K1": 178.91, "469ED1F C1KS": 383.89, "469WC1F C1SK": 366.80, "660073": 36.80,
    "2634": 10.61, "20940LPSKDS1K1": 176.98, "44028GPSKDA1K1": 178.76, "44079GPSKDA1K2": 178.76,
    "44090LPSKDA1K1": 177.10, "44091LPSKDA1K1": 176.98, "44057GPSKDA1K1": 176.98, "MBSS1360 B2SX": 153.04,
    "20890LPSKDA1K1": 178.75, "20769GPSKPA1K1": 188.08, "20892GPSKDA1K1": 178.75, "20964LPSKDA1K1": 178.75,
    "20968GPSKDA1K1": 178.75, "20980GPSKDA1K1": 178.75, "20982G0SKNA2K1": 170.98, "20982GPSKDA1K1": 178.75,
    "20983GPSKDA1K1": 178.75, "20983GPSKDA1K2": 178.75, "20984GPSKDA1K1": 178.75, "35014GPSKDA1K1": 178.75,
    "35015GPSKDA1K1": 178.75, "44009GPSKDA1K1": 178.75, "44036LPSKDA1K1": 178.75, "44037LPSKDA1K1": 178.75,
    "44038LPSKDA1K1": 178.75, "44039LPSKDA1K1": 178.75, "44041GPSKDA1K1": 178.75, "77114LPSKBS2K1": 178.75,
    "77114LPSKDS1K1": 178.75, "F49SS026 D1SX": 426.64, "F49SS027 E1SX": 426.64, "MBSS1445 E1SX": 324.05,
    "MBSS1453 E2SX": 195.79, "MBSS1446 E2SX": 204.34, "MBSS1452 D2SX": 195.79, "469SS068F P1SX": 409.54,
    "MBSS1443 D1SX": 324.04, "SK10726": 48.00, "SK10727": 48.00, "SK10919": 48.00, "SK11111": 48.00,
    "SK11112": 48.00, "SK11195": 48.00, "SK11689": 48.00, "SK40039": 48.00, "SK40083": 48.00,
    "SK40085": 48.00, "SK40103": 48.00, "SK40119": 48.00, "2115NCM/1P": 245.00, "2115NCN/1A": 179.55,
    "2115NCW/1Y": 179.55, "2350AI/1A": 213.75, "MOLIFEGAA/8C": 297.01, "MOLIFEGAC/8R": 297.01,
    "44133GPSKDA1K1": 178.75, "44145LPSKDA1K1": 178.75, "23696LPSVDB1": 186.30, "LRR4681L KN12 R1RX": 85.49,
    "2115TUV/1A": 213.75, "FBSS1170 S1SX": 118.84, "FGSS1217 S1KX": 153.04, "MBSS1451 P2SX": 195.80,
    "MBSS1367 P2SX": 118.84, "CH50033P": 217.30, "469SS068NH": 469.39, '44015GPSKDA1K1' : 176.98, 'CN28366H' : 130.77,
    'CN21005X' : 126.26, '20455LPSKDB1' : 184.68, 'LRG4721L KP21C1K' : 85.49, '99130GPMVDE4' : 89.00, 
    'XSWUQPI001' : 125.00, '6103B' : 13.88, 'DW-6900U-1DR' : 119.70, '99614LPMVDM1'	: 113.40, '77245L0SVNA3' : 123.12,
    '20944LPSKDS1K1' : 176.98, 'MBSS1446' : 204.34, 'MTSS1106 S1SR' : 118.84, 'CH40080D' : 153.34,
    '44013lpskda1' : 178.91, '44055LPSKDA1K1' : 176.98, 'LRR4724L KP62 S2RX' : 84.99, '6696' : 22.98,
    'MBSS1452' : 195.79
}

In [13]:
df_arquivo['Preço Unitário'] = df_arquivo['SKU'].map(preco_custo)

In [14]:
df_arquivo['Total Custo'] = df_arquivo['Preço Unitário'] * df_arquivo['Unidades']

In [15]:
df_arquivo

,Data da venda,Título do anúncio,SKU,Unidades,Receita por produtos (BRL),Receita por envio (BRL),Tarifa de venda e impostos,Tarifas de envio,Estado,Tarifa de Frete,Preço Unitário,Total Custo
0,1 de fevereiro de 2025 09:49 hs.,Seculus,23696LPSVDB1,1,439.0,0.00,83.41,-22.45,Entregue,22.45,186.30,186.30
2,2 de fevereiro de 2025 21:05 hs.,Orient,MBSS1154A P2SX,1,298.0,0.00,56.62,-22.45,Entregue,22.45,153.04,153.04
3,2 de fevereiro de 2025 21:16 hs.,Lince,LRR4724L KP62 S2RX,1,197.4,0.00,37.51,-22.45,Entregue,22.45,84.99,84.99
4,3 de fevereiro de 2025 03:45 hs.,Herweg,6103,3,83.7,0.00,28.38,0.00,Entregue,0.00,13.88,41.64
5,3 de fevereiro de 2025 19:04 hs.,Herweg,6102,1,40.7,15.46,11.18,-15.46,Entregue,0.00,17.21,17.21
...,...,...,...,...,...,...,...,...,...,...,...,...
81,27 de fevereiro de 2025 16:15 hs.,Herweg,6103,1,31.0,0.00,10.06,0.00,Processando no centro de distribuição,0.00,13.88,13.88
82,28 de fevereiro de 2025 06:28 hs.,Herweg,6103,1,31.0,0.00,10.06,0.00,Entregue,0.00,13.88,13.88
83,28 de fevereiro de 2025 11:09 hs.,Herweg,6103,1,31.0,0.00,10.06,0.00,Processando no centro de distribuição,0.00,13.88,13.88
84,28 de fevereiro de 2025 17:57 hs.,Herweg,6103,1,27.9,23.40,9.46,-23.40,Entregue,0.00,13.88,13.88


In [36]:
df_arquivo.to_excel('2025LaPrima_fevereiro202502.xlsx', index=False)